<pre>TMDB 결측치값중 크롤링 할 수 있는 데이터를 가져와서 데이터를 채운 파일</pre>

In [1]:
# 데이터가 부족한 경우 사용
# NaN값을 제거하면 데이터가 5302개이므로 클롤링을 통해 NaN값을 채울려고 한다.
import pandas as pd
import numpy as np

- 데이터 분석(결측치 분석) 감독, 예산, 국가, 장르

In [2]:
# title budget genres country main_charactor director vote_average vote_count
df = pd.read_csv('data/movie_data_original.csv',
                 names="title postpath openDate budget genres country main_charactor director time overview vote_average vote_count".split(),
                 skiprows=1)

In [4]:
# 예산이 표기된 api가 없다...일단 제거 (4858개)
NoBudget = df[df['budget'] == 0]
df = df.drop(NoBudget.index)
len(df)

5142

In [25]:
# 감독이 없는 데이터 4개
NoDirectors = df[df['director'] == '-']
len(NoDirectors)

4

In [27]:
# 감독이 없는 리스트 중 타이틀을 string -> dict로 그중 title name만 가져옴
# Ndirector 에 리스트로 저장
import ast
Ndirector = []
for NoDirector in df.loc[NoDirectors.index].title:
    for s in ast.literal_eval(NoDirector).values():
        Ndirector.append(s)
Ndirector

['Mr. Sadman',
 'The Collective: Movie',
 "The Visual Effects of 'Scary Movie 4'",
 'Dragon Ball Z: The Fall of Men']

In [ ]:
# Mr. Sadman 검색 결과 나오지 않음
# The Collective: Movie => 알레그잔데르 나나우 감독
# 아래 데이터는 IMDB에서 결과가 나옴.
# The Visual Effects of 'Scary Movie 4' => Matthew Asner
# Dragon Ball Z: The Fall of Men => 요한 포레

In [30]:
# 장르 4개
Nogenre = df[df['genres'] == '[]']
len(Nogenre)

4

In [31]:
# 감독이 없는 리스트 중 타이틀을 string -> dict로 그중 title name만 가져옴
# Ndirector 에 리스트로 저장
Ngenre = []
for Nog in df.loc[Nogenre.index].title:
    for g in ast.literal_eval(Nog).values():
        Ngenre.append(g)
Ngenre

['Mr. Sadman',
 "The Visual Effects of 'Scary Movie 4'",
 '몽키킹: 전설의 시작',
 'Die Taskforce-Nachbarn']

In [32]:
# Mr. Sadman 검색 결과 나오지 않음
# 아래 데이터는 IMDB에서 결과가 나옴.
# The Visual Effects of 'Scary Movie 4' => Documentary
# 몽키킹: 전설의 시작 => 액션 / 판타지
# Die Taskforce-Nachbarn => Comedy

In [83]:
# 국가 4개
Nocountry = df[df['country'] == '[]']
len(Nocountry)

13

In [84]:
Ncountry = []
for Noc in df.loc[Nocountry.index].title:
    for c in ast.literal_eval(Noc).values():
        Ncountry.append(c)
Ncountry

['Night Feeders',
 'Shamelessly She-Hulk',
 'Spider-Man: Convergence',
 'Mirreyes vs Godínez',
 'Mr. Sadman',
 'Perfection',
 'Perfection',
 'Red Hook',
 "The Visual Effects of 'Scary Movie 4'",
 '몽키킹: 전설의 시작',
 '인비저블걸',
 'Dragon Ball Z: The Fall of Men',
 'Die Taskforce-Nachbarn']

In [88]:
# Mr.Sadman 결과는 없음 삭제
del Ncountry[4]

In [95]:
# 국가가 없는 영화중 개봉시기가 없는 영화가 있다.
Cdate = []
for Noc in df.loc[Nocountry.index].title:
    date = str(df.loc[df.title == f"{Noc}"]['openDate'])
    Cdate.append(date[8:12])
Cdate

['2006',
 '2009',
 '2023',
 '2019',
 'NaN\n',
 '2022',
 '2022',
 '2022',
 '2006',
 '2022',
 '2019',
 '2015',
 '2022']

In [96]:
# Mr.Sadman 결과는 없음 삭제
del Cdate[4]

In [80]:
import requests
import json
from tqdm.notebook import tqdm

In [135]:
url = f"https://imdb-api.com/en/API/Title/k_n5qxjb05/tt1375666"
response = requests.get(url)
result = json.loads(response.text)

<pre>IMDB 하루에 100번 밖에 호출이 불가능 조심할것</pre>

In [141]:
# IMDB 사용 제목으로 검색후 ID값 받기
movieid = []
movie_country = {}
for idx, title in tqdm(enumerate(Ncountry)):
    url = f'https://imdb-api.com/en/API/SearchMovie/k_n5qxjb05/{title}'
    response = requests.get(url)
    result = json.loads(response.text)
    try:
        for res in result['results']:
            if res['description'] == f"({Cdate[idx]})":
                movieid.append(res[0]['id'])
        print(title, movieid[idx])
        for code in movieid:
            url = f"https://imdb-api.com/en/API/Title/k_n5qxjb05/{code}"
            response = requests.get(url)
            result = json.loads(response.text)
            try:
                movie_country[title] = result['countries']
            except:
                print(title, " 의 국가가 나오지 않습니다.")
    except:
        print(title, " 의 id값이 없습니다.")

0it [00:00, ?it/s]

Night Feeders  의 id값이 없습니다.
Shamelessly She-Hulk  의 id값이 없습니다.
Spider-Man: Convergence  의 id값이 없습니다.
Mirreyes vs Godínez  의 id값이 없습니다.
Perfection  의 id값이 없습니다.
Perfection  의 id값이 없습니다.
Red Hook  의 id값이 없습니다.
The Visual Effects of 'Scary Movie 4'  의 id값이 없습니다.
몽키킹: 전설의 시작  의 id값이 없습니다.


KeyboardInterrupt: 

In [142]:
# Night Feeders => USA
# Shamelessly She-Hulk => USA
# Spider-Man: Convergence(2012) => USA
# Mirreyes vs Godínez => 멕시코
# Mr. Sadman => 결과값 x
# Perfection => USA
# Perfection => USA
# Red Hook => USA
# The Visual Effects of 'Scary Movie 4' => USA
# 몽키킹: 전설의 시작 => China
# 인비저블걸 => USA
# Dragon Ball Z: The Fall of Men => France
# Die Taskforce-Nachbarn => German

- 클롤링(네이버 사용X)

In [11]:
# 네이버 api에서 제목 검색 => 실패
with open('APIKey/Naver.json') as file:
    api = json.load(file)
client_id = api['id']
client_secret = api['secret']

In [12]:
search = "Chip and Potato: Chip's Holiday"
url = "https://openapi.naver.com/v1/search/movie.json"
headers = {
            'X-Naver-Client-Id': client_id,
            'X-Naver-Client-Secret': client_secret
           }
params = {
    'query' : '[극장판] 레이디버그: 미라클스톤의 비밀'
}
response = requests.get(url, headers=headers, params=params)
response

<Response [200]>

In [13]:
result = json.loads(response.text)
result

{'lastBuildDate': 'Fri, 26 Aug 2022 14:12:09 +0900',
 'total': 1,
 'start': 1,
 'display': 1,
 'items': [{'title': '<b>극장판</b> <b>레이디</b><b>버그</b>: <b>미라클</b><b>스톤</b>의 <b>비밀</b>',
   'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=171727',
   'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/1717/171727_P14_110648.jpg',
   'subtitle': 'Miraculous seasons 2',
   'pubDate': '2017',
   'director': '토마스 아스트뤽|',
   'actor': '여민정|남도형|홍시호|',
   'userRating': '9.39'}]}